# COVID-19 Forecasting: ARIMA + Prophet

Прогнозируем развитие эпидемии на будущее

Используем несколько методов и сравниваем точность

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')

# загружаем данные
df = pd.read_csv('data/covid_clean.csv')
df['Date'] = pd.to_datetime(df['Date'])

print('Все библиотеки загружены')
print(f'Данные: {df.shape}')

## 1. ARIMA модель

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# берем Россию для примера
russia = df[df['Country'] == 'Russia'].sort_values('Date').copy()
russia = russia.set_index('Date')

# разделяем на train и test
# последние 30 дней оставляем на тестирование
train_size = len(russia) - 30
train = russia['Confirmed'][:train_size]
test = russia['Confirmed'][train_size:]

print(f'Train размер: {len(train)}')
print(f'Test размер: {len(test)}')

# обучаем ARIMA модель
# (p,d,q) параметры подбираем вручную
# p=5 (авторегрессия), d=1 (дифференцирование), q=2 (скользящее среднее)
try:
    model_arima = ARIMA(train, order=(5, 1, 2))
    fitted_arima = model_arima.fit()
    print('\nARIMA модель обучена!')
    print(fitted_arima.summary())
except:
    print('Ошибка при обучении ARIMA, но это нормально')
    # sometimes ARIMA fails, что это нормально с синтетическими данными

In [ ]:
# делаем прогноз
try:
    # прогнозируем на test set
    forecast_arima = fitted_arima.get_forecast(steps=len(test))
    forecast_df = forecast_arima.conf_int(alpha=0.05)
    forecast_df['forecast'] = forecast_arima.predicted_mean
    
    # считаем метрики
    mae = mean_absolute_error(test, forecast_df['forecast'])
    rmse = np.sqrt(mean_squared_error(test, forecast_df['forecast']))
    mape = mean_absolute_percentage_error(test, forecast_df['forecast'])
    
    print(f'\nARIMA метрики:')
    print(f'MAE: {mae:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'MAPE: {mape:.2f}%')
    
    # визуализация
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # train
    ax.plot(train.index, train.values, label='Train', color='steelblue', linewidth=2)
    # test
    ax.plot(test.index, test.values, label='Test', color='green', linewidth=2)
    # прогноз
    ax.plot(test.index, forecast_df['forecast'], label='ARIMA Forecast', color='red', linewidth=2, linestyle='--')
    
    # доверительный интервал
    ax.fill_between(test.index,
                     forecast_df.iloc[:, 0],
                     forecast_df.iloc[:, 1],
                     color='red', alpha=0.2, label='95% CI')
    
    ax.set_xlabel('Дата')
    ax.set_ylabel('Количество случаев')
    ax.set_title('ARIMA прогноз COVID-19 в России')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('plots/11_arima_forecast.png', dpi=300, bbox_inches='tight')
    plt.show()
    
except Exception as e:
    print(f'Ошибка при прогнозировании: {e}')

## 2. Exponential Smoothing

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

try:
    # простое экспоненциальное сглаживание
    model_exp = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=30)
    fitted_exp = model_exp.fit(optimized=True)
    
    # прогноз
    forecast_exp = fitted_exp.forecast(steps=len(test))
    
    # метрики
    mae_exp = mean_absolute_error(test, forecast_exp)
    rmse_exp = np.sqrt(mean_squared_error(test, forecast_exp))
    
    print(f'Exponential Smoothing метрики:')
    print(f'MAE: {mae_exp:.2f}')
    print(f'RMSE: {rmse_exp:.2f}')
    
    # график
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(train.index, train.values, label='Train', color='steelblue', linewidth=2)
    ax.plot(test.index, test.values, label='Actual', color='green', linewidth=2)
    ax.plot(test.index, forecast_exp, label='Exponential Smoothing', color='purple', linewidth=2, linestyle='--')
    
    ax.set_xlabel('Дата')
    ax.set_ylabel('Количество случаев')
    ax.set_title('Exponential Smoothing прогноз')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('plots/12_exponential_smoothing.png', dpi=300, bbox_inches='tight')
    plt.show()

except Exception as e:
    print(f'Ошибка при exponential smoothing: {e}')
    mae_exp = float('inf')
    rmse_exp = float('inf')

## 3. Prophet (Facebook)

In [ ]:
from prophet import Prophet

# Prophet требует специальный формат данных
russia_prophet = pd.DataFrame({
    'ds': russia.index,
    'y': russia['Confirmed'].values
})

# обучаем Prophet
# Prophet автоматически обнаруживает тренды и сезонность
model_prophet = Prophet(yearly_seasonality=True, daily_seasonality=False, interval_width=0.95)
model_prophet.fit(russia_prophet)

print('Prophet модель обучена!')

# прогноз
future = model_prophet.make_future_dataframe(periods=len(test))
forecast_prophet = model_prophet.predict(future)

# берем только часть для test
forecast_test = forecast_prophet.iloc[-len(test):]

# метрики
mae_prophet = mean_absolute_error(test.values, forecast_test['yhat'].values)
rmse_prophet = np.sqrt(mean_squared_error(test.values, forecast_test['yhat'].values))
mape_prophet = mean_absolute_percentage_error(test.values, forecast_test['yhat'].values)

print(f'\nProphet метрики:')
print(f'MAE: {mae_prophet:.2f}')
print(f'RMSE: {rmse_prophet:.2f}')
print(f'MAPE: {mape_prophet:.2f}%')

In [ ]:
# график Prophet
fig, ax = plt.subplots(figsize=(14, 6))

# train data
ax.plot(russia.index, russia['Confirmed'], label='Actual', color='steelblue', linewidth=2)

# forecast
ax.plot(forecast_prophet['ds'], forecast_prophet['yhat'], label='Prophet Forecast', color='red', linewidth=2)

# доверительный интервал
ax.fill_between(forecast_prophet['ds'],
                  forecast_prophet['yhat_lower'],
                  forecast_prophet['yhat_upper'],
                  color='red', alpha=0.2, label='95% CI')

# вертикальная линия где заканчивается обучение
ax.axvline(x=train.index[-1], color='black', linestyle='--', alpha=0.5, label='Train/Test split')

ax.set_xlabel('Дата')
ax.set_ylabel('Количество случаев')
ax.set_title('Prophet прогноз COVID-19')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/13_prophet_forecast.png', dpi=300, bbox_inches='tight')
plt.show()

## Сравнение методов

In [ ]:
# создаем таблицу сравнения
comparison = pd.DataFrame({
    'Method': ['ARIMA', 'Exponential Smoothing', 'Prophet'],
    'MAE': [mae, mae_exp, mae_prophet],
    'RMSE': [rmse, rmse_exp, rmse_prophet]
})

print('\nСравнение методов прогнозирования:')
print(comparison)

# график сравнения
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

comparison.set_index('Method')['MAE'].plot(kind='bar', ax=axes[0], color=['steelblue', 'orange', 'green'])
axes[0].set_title('Сравнение MAE')
axes[0].set_ylabel('MAE')
axes[0].tick_params(axis='x', rotation=45)

comparison.set_index('Method')['RMSE'].plot(kind='bar', ax=axes[1], color=['steelblue', 'orange', 'green'])
axes[1].set_title('Сравнение RMSE')
axes[1].set_ylabel('RMSE')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('plots/14_models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# сохраняем результаты
comparison.to_csv('results/forecast_comparison.csv', index=False)
print('\nРезультаты сохранены')

## Долгосрочный прогноз

In [ ]:
# делаем долгосрочный прогноз на 6 месяцев вперед
future_long = model_prophet.make_future_dataframe(periods=180)  # 180 дней = ~6 месяцев
forecast_long = model_prophet.predict(future_long)

# график
fig, ax = plt.subplots(figsize=(14, 7))

# исторические данные
ax.plot(russia.index, russia['Confirmed'], label='Исторические данные', color='steelblue', linewidth=2)

# прогноз
ax.plot(forecast_long['ds'], forecast_long['yhat'], label='Прогноз', color='red', linewidth=2)

# интервал
ax.fill_between(forecast_long['ds'],
                 forecast_long['yhat_lower'],
                 forecast_long['yhat_upper'],
                 color='red', alpha=0.2, label='95% доверительный интервал')

# линия где начинается прогноз
ax.axvline(x=russia.index[-1], color='black', linestyle='--', alpha=0.5)

ax.set_xlabel('Дата')
ax.set_ylabel('Количество случаев')
ax.set_title('Долгосрочный прогноз COVID-19 (6 месяцев)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/15_long_term_forecast.png', dpi=300, bbox_inches='tight')
plt.show()

print('Долгосрочный прогноз выполнен!')